---
title: Magnetic field data pipeline
---

In additional to the general data pipeline, we add feature extraction to the pipeline...

The product of this pipeline is a data set of interesting magnetic field events.

In [ ]:
#| hide
#| export
from kedro.pipeline import Pipeline, node
from kedro.pipeline.modular_pipeline import pipeline

from ids_finder.utils.basic import load_params
from ids_finder.pipelines.default.data import (
    create_pipeline_template as create_pipeline_template_base,
)

from typing import Callable, Optional

In [ ]:
#| default_exp pipelines/default/data_mag

In [ ]:
# | export
def create_pipeline_template(
    sat_id: str,  # satellite id, used for namespace
    source: str,  # source data, like "mag" or "plasma"
    extract_features_fn: Optional[Callable] = None,
    **kwargs,
) -> Pipeline:
    params = load_params()
    sat_id = sat_id.upper()
    source = source.upper()

    namespace = f"{sat_id}.{source}"

    tau = params["tau"]
    ts = params[sat_id][source]["time_resolution"]
    tau_str = f"tau_{tau}s"
    ts_str = f"ts_{ts}s"

    node_extract_features = node(
        extract_features_fn,
        inputs=[f"primary_data_{ts_str}", "params:tau", "params:extract_params"],
        outputs=f"feature_{tau_str}",
        name="extract_features",
    )

    nodes = [node_extract_features]

    pipelines = pipeline(nodes, namespace=namespace)

    base_pipelines = create_pipeline_template_base(
        sat_id=sat_id, source=source, **kwargs
    )
    return base_pipelines + pipelines